# Algoritmos de optimización - Trabajo Práctico
### Nombre y Apellidos: Mikel Escobar de Carlos 
Url: https://github.com/Mikelesc/03MIAR_Algoritmos-de-Optimizacion

Collab: https://colab.research.google.com/drive/1ivbqQjGT82nGU6wZAsbRAEKWqnYPp1s-?usp=sharing



## Problema 3: Combinar cifras y operaciones

Descripción del problema:

**Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las
operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/). Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una cantidad
dada. Un ejemplo sería para obtener el 4:**

**4+2-6/3*1 = 4**



                                        

### *¿Cuantas posibilidades hay sin tener en cuenta las restricciones? ¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.

Sin tener en cuenta las restricciones podemos repetir tanto números como operaciones, luego tenemos 13 elementos (9 cifras y 4 operaciones) y en caso de que tampoco haya restricción en el número de elementos a utilizar habría **infinitas** combinaciones. Si consideramos que el número máximo de elementos sigue siendo 9, pero con repetición y sin importar si es una cifra o una operación, entonces tendremos el número de variaciones con repetición. Cada vez que escojamos uno de los 13 elementos podemos combinarlo con uno cualquiera de los 13, luego tendríamos **13^9 posibilidades**, aunque esto daría lugar a expresiones sin sentido como tener 9 signos de multiplicación. Para que tuviese más sentido podríamos imponer que haya que intercalar cifras y operaciones, pero en ese caso estamos ya casi resolviendo el problema con restricciones.



Teniendo en cuenta todas las restricciones sí que tenemos un espacio de posibilidades acotado. Al no poder repetir ni cifras ni operadores y tener que combinarlos de forma alterna, tendremos las posibles variaciones sin repetición de cifras multiplicadas por las posibles variaciones sin repetición de operaciones:

$V_m^n= m ·(m-1) ·(m-2) ··· (m - n + 1) = \frac{m!}{(m-n)!}$

* Las variaciones de las 4 operaciones cogiendo las 4 eqivalen a : $4!$

* Las variaciones de las 9 cifras cogiendo 5 de ellas equivalen a: $9!/4!$

Luego en total tenemos:

Posibilidades = $\frac{9!}{4!}\cdot\frac{4!}{0!}$ = 9! = **362880**


### * ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, argumentalo)


Bueno así a priori lo que veo es que tenemos dos listas de elementos. Una lista (o un set ya que no repetimos elementos) con las 9 cifras y otra lista con las 4 operaciones. De momento comenzaré con esta estructura de datos.

### *¿Cual es la función objetivo?

Buscamos construir una expresión aritmética cuya evaluación sea igual a la cantidad dada, es decir eval(expresion) - cantidad = 0


### *¿Es un problema de maximización o minimización?

De primeras parece un problema de búsqueda, no estoy maximizando ni minimizando una función, simplemente busco (si existe) la combinación que cumple las restricciones.

### Diseña un algoritmo para resolver el problema por fuerza bruta

Por fuerza bruta podemos simplemente realizar todas las combinaciones posibles y evaluarlas hasta encontrar aquella que satisfaga todas las condiciones. Vamos a echar mano de itertools para calcularlas todas. El método permutations recible una lista de elementos de tamaño m y devuelve combinaciones de tamaño n de esos elementos sin repetición.

In [6]:
from itertools import permutations
from functools import reduce
import operator

In [168]:
%%time
cifras = ["1","2","3","4","5","6","7","8","9"]
operaciones = ["+","-","*","/"]

def fuerza_bruta(objetivo, cifras = cifras, operaciones = operaciones):
    for numeros in permutations(cifras, len(operaciones)+1):
        for signos in permutations(operaciones):
            expresion_lst = [x+y for x, y in zip([""] + list(signos),numeros)]
            expresion = "".join(expresion_lst)
            if eval(expresion) == objetivo:
                return expresion
    return f"No se ha encontrado expresión para {objetivo}"

fuerza_bruta(65)

Wall time: 1.79 s


'4/1-2+7*9'

Lo podemos modificar brevemente para analizar cuales son los valores máximo y mínimo que podemos obtener:

In [4]:
%%time
def fuerza_bruta_max_min(cifras = cifras, operaciones = operaciones):
    lista = []
    for numeros in permutations(cifras, len(operaciones)+1):
        for signos in permutations(operaciones):
            expresion_lst = [x+y for x, y in zip([""] + list(signos),numeros)]
            expresion = "".join(expresion_lst)
            lista.append(eval(expresion))
    return max(lista), min(lista)

fuerza_bruta_max_min()

Wall time: 3.68 s


(78.83333333333333, -70.71428571428571)

### Calcula la complejidad del algoritmo por fuerza bruta

Como vemos vamos recorriendo todas las combinaciones posibles tal que:

$\frac{m!}{(m-(n+1))!} * n!$   (En este caso m = 9 y n = 4)

Podríamos estimar que la complejidad es $O(m!·n!)$. En este caso especifico. Para un algoritmo con este enfoque a un problema que busque combinaciones en una lista de elementos podemos simplemente decir: $O(N!)$, donde N será el tamaño de la lista.

En nuestro problema, si tuviesemos conjuntos de tamaño similar podríamos decir: $O(n!^2)$ ya que tenemos dos listas.

A primera vista uno puede caer en pensar que es $O(n^2)$  ya que simplemente estamos recorriendo dos listas en el código (dos bucles anidados), pero el método **permutations** está haciéndonos el trabajo de calcular todas las combinaciones.
 

### *Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Vemos que le cuesta encontrar la solución desde 80ms a unos 3.6s en los casos más desfavorables (cuando tiene que evaluar casi todas las combinaciones) al algoritmo de fuerza bruta. De primeras me viene la idea de probar con un algoritmo de **ramificación y poda**, ya que podemos ir evaluando como nos acercamos al objetivo y descartar ramas que se alejan mucho. Por ejemplo si queremos expresar el -24 y tenemos un nodo que empieza por "8*9" puedes descartarlo.

In [2]:
from random import sample

cifras = ["1","2","3","4","5","6","7","8","9"]
operaciones = ["+","-","*","/"]

def generar_nodo(padre = None, cifras = cifras, operaciones = operaciones):
    """
    Dado un nodo añade una operación y una cifra de las que quedan.
    """
    if padre is None:
        return sample(operaciones,1)[0].join(sample(cifras,2))
    else:
        op_left = list(set(operaciones) - set(padre) - set(cifras))
        cif_left = list(set(cifras) - set(padre) - set(operaciones))
        if len(op_left) >= 1:
            return padre + sample(op_left,1)[0] + (sample(cif_left,1))[0]
        else:
            return padre
        
def calcular_cotas(nodo, cifras = cifras, operaciones = operaciones):
    """
    Calculamos cotas aproximadas.
    """
    if eval(nodo) > 0:
        op_max = ["*","+"]
        op_min = ["/","-"]
    else:
        op_max = ["/","+"]
        op_min = ["*","-"]
        
    op_left_max = list(set(op_max) - set(nodo))
    op_left_min = list(set(op_min) - set(nodo))
    cif_left = list(set(cifras)-set(nodo))
    if len(op_left_max) > 0:
        cota_max = eval(nodo + op_left_max[0] + max(cif_left))
    else:
        cota_max = eval(nodo)
        
    if len(op_left_min) > 0:
        cota_min = eval(nodo + op_left_min[0] + max(cif_left))
    else:
        cota_min = eval(nodo)
        
    return cota_max, cota_min
        

def evaluar(nodo, lista, objetivo):
    """
    Decidimos si continuar con ese nodo o no. Si es no se añade a la lista de ya explorados.
    """

    cota_max, cota_min = calcular_cotas(nodo)
    
    if cota_max > objetivo and cota_min < objetivo:
        print("Continuar, cotas: ", cota_max, cota_min)
        return True, lista

    else:
        print("Atrás, cotas: ", cota_max, cota_min)
        lista.append(nodo)
        return False, lista


def ramif_y_poda(objetivo, nodo=None, lista = [], cifras = cifras, operaciones = operaciones, contador_repeticiones = 0):
    """
    Funcion principal recursiva
    """
    if contador_repeticiones > 7: #Si no encontramos la solucion por ese camino volvemos a empezar
        return ramif_y_poda(objetivo = objetivo, nodo=None, lista = lista, cifras = cifras, operaciones = operaciones, contador_repeticiones = 0)
        
    # 1. Generar nodo a partir del que llega.
    nuevo_nodo = generar_nodo(nodo)
    print("Nuevo nodo: ", nuevo_nodo)
    
    if nuevo_nodo in lista: # Si está en la lista no lo exploramos
        contador_repeticiones += 1
        return ramif_y_poda(objetivo = objetivo, nodo = nodo, lista = lista, cifras = cifras, operaciones = operaciones, contador_repeticiones = contador_repeticiones)
        
    if len(nuevo_nodo) == 9: # Si ya tenemos los 9 elementos evaluamos si es la solucion
        if eval(nuevo_nodo) == objetivo:
            return nuevo_nodo
        else: # Si no es se añade a la lista y se vuelve atrás
            lista.append(nuevo_nodo) 
            return ramif_y_poda(objetivo = objetivo, nodo = nodo, lista = lista, cifras = cifras, operaciones = operaciones, contador_repeticiones = contador_repeticiones)
    
    # Si el nodo no tiene todos los elementos se evaluan sus cotas y se decide si continuar o volver atrás
    continuar, lista = evaluar(nuevo_nodo, lista, objetivo)
    if continuar:
        return ramif_y_poda(objetivo = objetivo, nodo = nuevo_nodo, lista = lista, cifras = cifras, operaciones = operaciones, contador_repeticiones = contador_repeticiones)
    else:
        return ramif_y_poda(objetivo = objetivo, nodo = nodo, lista = lista, cifras = cifras, operaciones = operaciones, contador_repeticiones = contador_repeticiones)
       

In [159]:
%%time
 
cifras = ["1","2","3","4","5","6","7","8","9"]
operaciones = ["+","-","*","/"]

resultado = ramif_y_poda(-14, nodo=None, lista = [], cifras = cifras, operaciones = operaciones, contador_repeticiones = 0)
print("\nRESULTADO", resultado)

Nuevo nodo:  9*4
Atrás, cotas:  44 28
Nuevo nodo:  7-9
Continuar, cotas:  6 -65
Nuevo nodo:  7-9/6
Atrás, cotas:  13.5 5.5
Nuevo nodo:  7-9+3
Atrás, cotas:  22 -1.625
Nuevo nodo:  7-9*3
Continuar, cotas:  -12 -20
Nuevo nodo:  7-9*3+1
Atrás, cotas:  -19.875 -19
Nuevo nodo:  7-9*3/2
Atrás, cotas:  1.5 -6.5
Nuevo nodo:  7-9*3/4
Atrás, cotas:  8.25 0.25
Nuevo nodo:  7-9*3+2
Atrás, cotas:  -19.75 -18
Nuevo nodo:  7-9*3+8
Atrás, cotas:  -18.666666666666668 -12
Nuevo nodo:  7-9*3+5
Atrás, cotas:  -19.375 -15
Nuevo nodo:  7-9*3/4
Nuevo nodo:  7-9*3+1
Nuevo nodo:  7-9*3/5
Atrás, cotas:  9.6 1.5999999999999996
Nuevo nodo:  7-9*3+5
Nuevo nodo:  7-9*3+8
Nuevo nodo:  7-9*3+4
Atrás, cotas:  -19.5 -16
Nuevo nodo:  7-9*3+6
Atrás, cotas:  -19.25 -14
Nuevo nodo:  7-9*3+5
Nuevo nodo:  7-9*3+6
Nuevo nodo:  7-9*3/1
Continuar, cotas:  -12.0 -20.0
Nuevo nodo:  7-9*3/1+2
Nuevo nodo:  7-9*3/1+6

RESULTADO 7-9*3/1+6
Wall time: 8.98 ms


Al principio pensaba que podía fallar esta aproximación ya que el hecho de comenzar con un nodo aleatorio e ir moviendote, aunque las cotas máxima y mínima estén por encima y debajo de tu objetivo respectivamente, para este problema en concreto puede no existir la solución. De hecho, al principio me daba error por exceso de recursividad ya que aunque tengo la lista de nodos ya explorados para evitarlos, simplemente se volvía a llamar la función con el mismo nodo y se repetían sin que se pudiese salir de ese primer nodo que era incompatible con la solución. El remedio a ese problema ha sido colocar un contador, y si veo que se ha quedado atascado con ese nodo inicial y en esa rama simplemente vuelvo a empezar con otro nodo.




Y como vemos...
#### Seguro que hay formas más eficientes, pero funciona sorprendentemente bien!!!!

**Recordamos que el método de fuerza bruta tenía un mínimo de 80ms en las situaciones más favorables y 4s en las más desfavorables. En el caso de mi solución vemos que si tenemos suerte al elegir el primer nodo podemos llegar a la solución en <10ms y en casos con menos suerte unos 100-200ms. Esto es una mejora sustancial!** 


Una de las mejoras claras que veo es en la función de determinar las cotas. Desde luego no es lo ideal ya que puede ser que con otro orden de operadores ese nodo sea también llegue a la solución. Por el momento he usado esa aproximación, que podemos considerar casi voraz, donde tomamos que para un nodo positivo si lo sumo o lo multiplico por el máximo de las cifras restantes esta es su cota máxima y si lo divido o resto por esa misma cifra es su cota mínima. Es una primera aproximación que como vemos funciona. He pensado en mejorarlo empleando una forma más sofisticada.

También soy consciente de que no es un algoritmo de ramificación y poda "de libro". Realmente tiene cierta parte de búsqueda heurística, ya que no exploro y acoto todos los nodos hijos de cada nodo, es decir no estoy podando un árbol completo ya que soy consciente de que puede no estar la solución en el mismo. Por ello he introducido la parte de deshechar un nodo rápidamente si veo que se alarga la búsqueda con él y he introducido un componente aleatorio en la generación de los nodos.


### *Calcula la complejidad del algoritmo 

No estoy tan seguro de cómo ha mejorado la complejidad. No tenemos un algoritmo determinista, lo rápido o lento que encontremos la solución va a depender de la suerte que tengamos escogiendo los nodos. Lo bueno es que tenemos la lista de todas aquellas posibilidades que ya hemos probado, y si un nodo inicial no ha funcionado ya no volveremos a intentarlo.

Dado que vamos eliminando posibilidades de nodos iniciales y estamos explorando un árbol creo que la complejidad es **exponencial**, que es el tipo de complejidades que vemos en otros algoritmos recursivos. $O(m^n)$
 

### Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

No creo que en este caso aplique esta pregunta. Si mantenemos las restricciones de número de cifras y operaciones el problema es el mismo.

### Aplica el algoritmo al juego de datos generado

Voy a añadir algo como un operador exponencial y probamos, lo hago al final del cuaderno ya que no entra dentro del objetivo trabajo realmente y asi es más fácil de seguir lo que importa de cara a la evaluación.

### Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

itertools permutations: https://docs.python.org/3/library/itertools.html#itertools.permutations

### Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Como ya he comentado mi primer paso para mejorar sería cambiar el método de calcular las cotas por algo más sofisticado.

Otra de las aproximaciones a resolverlo que quizá mejore el rendimiento podría ser un algoritmo genético, donde fuesemos creando candidatos y cruzándolos y mutándolos. Al final la forma de modelar un individuo en este problema es muy sencilla ya que simplemente es un string que cumple una serie de normas, y la función de evaluación es aún más sencilla con el método eval(). 

Se podría complicar mucho más el problema añadiendo más números, aunque sean de dos cifras, o decimales. Y sobre todo sería muy sencillo añadir operaciones como la exponencial o la raíz cuadrada. De ese modo tendríamos un espacio de soluciones muchísimo más grande y por fuerza bruta sería mucho más costoso y probablemente inabarcable con un ordenador doméstico.

## Probamos a añadir datos de entrada más complejos


He probado añadiendo el operador exponencial, pero esta es una operación de por sí costosa para la cpu y se extiende demasiado en el tiempo. Voy a probar a añadir simplemente la cifra 1/2. Además para añadir el exponencial he tenido que modificar la función de calcular cotas y añadir calculos de exponenciales en el proceso lo hace mucho más complejo para la cpu.

Veamos que pasa añadiendo el 0.5:

In [24]:
%%time
cifras = ["1","2","3","4","5","6","7","8","9","0.5"]
operaciones = ["+","-","*","/"]

resultado = ramif_y_poda(34.5, nodo=None, lista = [], cifras = cifras, operaciones = operaciones, contador_repeticiones = 0)
print("\nRESULTADO", resultado)

Nuevo nodo:  6+0.5
Atrás, cotas:  10.5 6.055555555555555
Nuevo nodo:  6+2
Atrás, cotas:  24 6.222222222222222
Nuevo nodo:  5/2
Atrás, cotas:  22.5 -6.5
Nuevo nodo:  1-5
Atrás, cotas:  0.4444444444444444 -44
Nuevo nodo:  1*0.5
Atrás, cotas:  9.5 0.05555555555555555
Nuevo nodo:  5/2
Nuevo nodo:  7*1
Atrás, cotas:  16 0.7777777777777778
Nuevo nodo:  2+3
Atrás, cotas:  29 2.3333333333333335
Nuevo nodo:  6*3
Atrás, cotas:  27 2.0
Nuevo nodo:  7+1
Atrás, cotas:  16 7.111111111111111
Nuevo nodo:  0.5+5
Continuar, cotas:  45.5 1.0555555555555556
Nuevo nodo:  0.5+5-6
Atrás, cotas:  4.833333333333333 -48.5
Nuevo nodo:  0.5+5/6
Atrás, cotas:  8.0 -7.666666666666666
Nuevo nodo:  0.5+5-2
Atrás, cotas:  -12.5 5.277777777777778
Nuevo nodo:  0.5+5/0.5
Nuevo nodo:  0.5+5/7
Atrás, cotas:  6.928571428571429 -7.785714285714286
Nuevo nodo:  0.5+5*1
Atrás, cotas:  5.5 1.0555555555555556
Nuevo nodo:  0.5+5/9
Atrás, cotas:  4.944444444444445 -6.944444444444445
Nuevo nodo:  0.5+5-3
Atrás, cotas:  -21.5 5.16666

Vemos así que el algoritmo funciona perfectamente añadiendo elementos a los datos de entrada. Sin embargo como ya he dicho añadir una operación nueva como la exponencial requiere modificar también la función de calcular cotas y le toma mucho tiempo terminar.

In [ ]:
cifras = ["1","2","3","4","5","6","7","8","9"]
# operaciones = ["+","-","*","/", "**"]

resultado = ramif_y_poda(240, nodo=None, lista = [], cifras = cifras, operaciones = operaciones, contador_repeticiones = 0)
print("\nRESULTADO", resultado)